# Bumblekite Tutorial 1: Hidden Hypoxemia's Impact on In-Hospital Mortality


* Understanding the Problem: Pulse oximeters are medical devices used to measure the level of oxygen in the blood without needing a blood sample. The most accurate method, however, requires taking a blood sample to measure oxygen levels directly. Pulse oximeters currently available are less accurate in people with darker skin tones. Theses pulse oximetry inaccuracies can fail to detect episodes of hidden hypoxemia, i.e., low SaO2 with high SpO2. Hidden hypoxemias can result in less treatment and increased mortality. Yet flawed, pulse oximeters remain ubiquitously used because of their ease of use; debiasing the underlying algorithms could alleviate the downstream repercussions of hidden hypoxemia.

* Dataset: BOLD is a new comprehensive dataset that aims to underscore the importance of addressing biases in pulse oximetry accuracy, which disproportionately affect darker-skinned patients. The dataset was created by harmonizing three Electronic Health Record databases (MIMIC-III, MIMIC-IV, eICU-CRD) comprising Intensive Care Unit stays of US patients.

* Main Paper: The tutorial is build upon the paper titled "Analysis of Discrepancies Between Pulse Oximetry and Arterial Oxygen Saturation Measurements by Race and Ethnicity and Association With Organ Dysfunction and Mortality" published in JAMA Network Open. This paper investigates discrepancies between pulse oximetry (SpO2) and arterial oxygen saturation (SaO2) measurements, examining the impact of these discrepancies on clinical outcomes across different racial and ethnic groups.


* Objective of the tutorial: The objective of this tutorial is to provide a comprehensive guide on how to analyze and address biases in healthcare data, with a specific focus on hidden hypoxemia and its impact on in-hospital mortality predictions. Through a series of structured steps, including exploratory data analysis, data preprocessing, train-test splitting, and model evaluation, participants will learn how to:

1. Detect and understand hidden patterns and biases in healthcare datasets.
2. Preprocess data effectively to ensure robust and reliable model performance.
3. Evaluate the influence of hidden hypoxemia on patient outcomes using statistical and machine learning techniques.
4. Assess and compare the performance of different models across various racial groups to highlight the importance of careful feature selection and bias mitigation in clinical predictions.


* Key Variables to keep in mind:

1. **Pulse Oximetry (SpO2)**: This is a [ non-invasive ] method used to measure the oxygen level (oxygen saturation) in the blood. It is usually done using a device clipped onto a finger, toe, or earlobe. Normal SpO2 levels range from 95% to 100%, indicating sufficient oxygen in the blood. Levels below 95% can indicate hypoxemia, with severe hypoxemia occurring below 85%.
2. **Arterial Oxygen Saturation (SaO2)**: This is an [ invasive ] measurement of oxygen saturation directly from the blood using an arterial blood gas (ABG) test. It is more accurate than pulse oximetry but requires drawing blood from an artery. SaO2 provides a direct measurement of oxygen levels in the blood and is used to confirm the accuracy of SpO2 readings, especially in cases of suspected hypoxemia.
3. **ABG**: Arterial Blood Gas
4. **Hypoxemia**: This refers to low levels of oxygen in the blood. It can be dangerous and requires medical attention.
5. **Hidden Hypoxemia**: This occurs when a patient's pulse oximetry reading (SpO2) suggests they have normal oxygen levels, but their arterial oxygen saturation (SaO2) shows they actually have low oxygen levels. → (ie, SpO2 ≥ 88% but SaO2 <88%).
6. **Organ Dysfunction**: This refers to the impaired function of organs (like the heart, lungs, liver, or kidneys) often assessed using specific scores like the Sequential Organ Failure Assessment (SOFA) score.
7. **In-hospital Mortality**: This means death occurring during a hospital stay.
8. **Sequential Organ Failure Assessment (SOFA) Score**: A scoring system used to track a patient's status during their stay in an intensive care unit (ICU). Higher scores indicate more severe organ dysfunction.
9. **Electronic Health Record (EHR)**: Digital version of patients' paper charts. They contain the medical and treatment history of patients.


Note: This tutorial draws inspiration from earlier datathons, particularly the [MIT Critical Datathon 2023](https://github.com/CriticalDatathon) and the [MIT Brown Datathon 2024](https://github.com/joamats/mit-brown-datathon).


**👥 Team Names**

`Add your names here (alphabetically)`

- ....
- ....


# Loading the dependencies and libraries

In [ ]:
# If you are using Jupyter Notebook, make sure you are in the right environment kernel!

In [ ]:
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# Loading the Data

In [ ]:
# Mount Google Drive to access your files:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# After ensuring that your CSV file is placed in your Google Drive, run:

import pandas as pd

# df_initial = pd.read_csv("your_drive_path_to_file.csv")

df_initial = pd.read_csv("/content/drive/MyDrive/2024-2025/Summer_2024/BumbleKite_2024/Datathon/full_code/participants/bold_dataset.csv")
# for me the BOLD dataset is in: /content/drive/MyDrive/BumbleKite_2024/Tutorial_content/data/participants/bold_dataset.csv

In [ ]:
# Did it work? Inspect the Data:

# Peak at our data (First 5 rows)
df_initial.head()

In [ ]:
# Get a summary of the DataFrame
print(df_initial.info())

In [ ]:
# Get the column names
column_names = df_initial.columns.tolist()
print ("Column names:", column_names)

For an overview of all the variables and their meanings, please refer to the paper on the dataset [BOLD: Blood-gas and Oximetry Linked Dataset](https://www.nature.com/articles/s41597-024-03225-z#Sec23) . Check the "Description of fields" section for detailed information.


In [ ]:
# List of key variables to explore further
key_variables = [

    'source_db', 'comorbidity_score_value',

    # ids
    'unique_subject_id', 'unique_hospital_admission_id', 'unique_icustay_id',

    # patient
    'admission_age', 'sex_female', 'race_ethnicity', 'BMI_admission', # BMI: Body Mass Index
    'weight_admission' , 'height_admission',

    # ICU stay info
    'sofa_past_overall_24hr', 'sofa_future_overall_24hr',

    # Vital signs
    'vitals_heart_rate',  'vitals_mbp_ni', 'vitals_resp_rate', 'vitals_tempc',
        ## MBP: Mean Blood Measure
        ## invasive(i) and non invasive(ni)
        ## DBP: Diastolic Blood Pressure

    # LABS
    'cbc_wbc', 'cbc_hematocrit', 'cbc_platelet',
    'coag_fibrinogen', 'coag_inr',
    'bmp_sodium', 'bmp_potassium', 'bmp_chloride', 'bmp_bicarbonate', 'bmp_bun', 'bmp_creatinine',
    'bmp_glucose', 'bmp_aniongap', 'bmp_calcium', 'bmp_lactate', 'hfp_alt', 'hfp_alp', 'hfp_ast',
    'hfp_bilirubin_total', 'hfp_albumin', 'others_ck_cpk',
    'others_ld_ldh',

    # Target
    'in_hospital_mortality',

     # ABG data
    'pH',

    # Study Features
    'SpO2', 'SaO2'

]

In [ ]:
# Filter the dataset to only focus on the key variables relevant to the tutorial
df_key_variables = df_initial [key_variables]

In [ ]:
# Visualize the columns of the new dataframe
print(df_key_variables.columns.tolist())

In [ ]:
df_key_variables.shape

In [ ]:
# Calculate the percentage of each unique value in the 'race_ethnicity' column
race_ethnicity_counts = df_key_variables['race_ethnicity'].value_counts(normalize=True) * 100

# Display the percentages
print(race_ethnicity_counts)

In [ ]:
# Filter the Dataframe to include only the most common racial/ethnic groups
selected_groups = ['Asian','Hispanic OR Latino', 'Black', 'White']
df_filtered = df_key_variables[df_key_variables ['race_ethnicity'].apply(lambda x: x in selected_groups)].copy()

In [ ]:
df_filtered.shape

# Train Test Split



In [ ]:
from sklearn.model_selection import train_test_split

data_train, data_test = train_test_split (df_filtered, test_size = 0.2, random_state = 42)

In [ ]:
print("Shape of the training data", data_train.shape)
print("Shape of the testing data", data_test.shape)

### Check balancing of the mortality outcome

* Question to the participants:
  - What is the purpose of this step?
  - How should we go about it?


* My Answer: The goal of checking the mean of hospital_death in the overall dataset, training set, and test set is to ensure that the distribution of the target variable (in-hospital mortality) is balanced across these subsets. This helps to verify that the data splitting process did not introduce any bias, ensuring the model will be trained and evaluated on representative samples.
[ ask them to fill their answer ]

In [ ]:
print(df_filtered.in_hospital_mortality.mean())
print(data_train.in_hospital_mortality.mean())
print(data_test.in_hospital_mortality.mean())

- Any observations?
They should not be too different as we need to ensure that the proportions of the target variable are similar across the overall dataset, training set, and test set helps in building a model that is more likely to generalize well and provides accurate performance metrics.

# Initial Exploratory Data Analysis (EDA)

## Pandas profiling

Pandas-profiling delivers an extended analysis of a DataFrame. Really helpful!

In [ ]:
# Inspect the data types of each column
print(data_train.dtypes)

In [ ]:
! pip install pandas-profiling

In [ ]:
! pip install ydata_profiling

In [ ]:
import os
from ydata_profiling import ProfileReport

# Path where the report should be saved
save_path = "/content/drive/MyDrive/2024-2025/Summer_2024/BumbleKite_2024/Datathon/pandas_reports/data_train_copy_report.html"

# Ensure the directory exists
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Generate the profile report
profile = ProfileReport(data_train, title="Pandas Profiling Report of the Training Data")

# To display the report in a Jupyter notebook
profile.to_notebook_iframe()

# Save the report to an HTML file
profile.to_file(save_path)

Note: You can download the HTML file from your Drive to your local computer and view it there!

#### The Profile Report will take around 10 mins to run, so while it is loading....


* Think of the question: Why do you think we are doing the train test split right after loading the data specifically? Why not before or after?
* My answer: To prevent data leakage and ensure the test set remains hidden, this step should be performed immediately after loading the data, before any exploratory data analysis (EDA) or preprocessing.



Now that we did our train test split, pre processing & EDA should be done solely on train set!

## Overview of the data using Tableone
Goal: To understand the distribution of patients across different racial-ethnic groups, sex, and other demographics. This step sets the foundation by showing the diversity in the dataset and highlighting potential areas of interest for further analysis.

In [ ]:
# Create a new column with 'F' for female and 'M' for male
data_train['sex'] = data_train['sex_female'].apply(lambda x: 'F' if x == 1 else 'M')

In [ ]:
! pip install tableone

In [ ]:
data_train_TableOne = data_train.copy()

In [ ]:
# Assuming your DataFrame is named `data_train_TableOne`
data_train_TableOne['in_hospital_mortality'] = data_train_TableOne['in_hospital_mortality'].astype(str)

In [ ]:
from tableone import TableOne

# Specify the columns to include in the table
columns = ['admission_age', 'BMI_admission', 'race_ethnicity', 'sex', 'in_hospital_mortality']

# Categorical Variables [represent categories]
categorical = ['sex', 'in_hospital_mortality']


# Define how to group the data for comparison
# In this case grouping by 'in_hospital_mortality' to compare different groups based on their mortality status
# Optional: Try to group by another outcome and see what you can get.
groupby = 'race_ethnicity'

# Indicates which variables are not normally distributed
nonnormal = ['BMI_admission']

# Renames the variables for better readability
labels = {
    'in_hospital_mortality': 'Mortality',
    'admission_age': 'Age',
    'BMI_admission': 'BMI',
    'sex': 'Sex'
}

# Sets the order for categorical variables
order = {
    'sex': ['F', 'M'],  # 1 for Female, 0 for Male
    'in_hospital_mortality': [0,1]
}



In [ ]:
TableOne_train_set = TableOne(
    data_train_TableOne,  # The dataset to summarize
    columns=columns,
    categorical=categorical,
    groupby=groupby,
    nonnormal=nonnormal,
    rename=labels,
    order=order,
    #limit=limit,
    overall=True,  # Whether to include overall summary statistics
    missing=True,  # Whether to include info about the number of missing values for each variable
    pval=True,     # Whether to include p-values for comparisons
    decimals=2     # Number of decimal places for numerical values
)


# Display the table
display(TableOne_train_set)

You can create your own TableOne table with the variables of your choice!


- Columns & Rows:
  * Missing: Number of missing values for each variable.
  * Overall: Shows the count and percentage of each subgroup in the entire dataset.
  * 0.0 (No Mortality): Shows the count and percentage of each subgroup within the group that did not experience mortality.
  * 1.0 (Mortality): Shows the count and percentage of each subgroup within the group that experienced mortality.
  * P-Value: Indicates whether there is a statistically significant difference between the distributions of the subgroups across the mortality categories.

- Variables:
  * admission_age, mean (SD): Mean age at admission with standard deviation (SD). SD measures the amount of variation or dispersion of ages.
  * BMI, median [Q1, Q3]: Median Body Mass Index (BMI) with the first quartile (Q1) and third quartile (Q3). Median represents the middle value, and quartiles divide the data into four equal parts.
  * Race and Ethnicity, n (%): Number and percentage of patients in different racial and ethnic groups.
  * Sex, n (%): Number and percentage of male and female patients.

- Grouped by Mortality:
  * Statistics for Subgroups: Compares characteristics of patients who survived (0.0) versus those who died (1.0) during their hospital stay.
  * P-Value: Indicates whether differences between the mortality groups are statistically significant. A low p-value (typically < 0.05) suggests significant differences.


- Observations:
  * In the overall dataset, Black individuals make up around 10%.
  * If you sum up all the percentages in the row of 0.0 for each separate demographic variable, you will get 100%.

### Interpretation of the table:


- Question: Can you comment on the table?
- Your Answer: ....

- My answer:
  * Data Representation and Bias: The disproportionate representation of white patients in the dataset might affect the detection of hypoxemia in non-white patients. Since pulse oximeters are known to be less accurate for individuals with darker skin tones, this imbalance could lead to underestimation of hypoxemia in underrepresented groups.

  * Clinical Implications: If hypoxemia is not accurately detected due to demographic bias in the dataset, non-white patients might receive suboptimal care. Building models predicting hypoxemia with this dataset without a consideration of the distribution will create models biased towards the white population and will not detect many cases in black patients and other minority races. This analysis highlights the need for diverse datasets to ensure equitable healthcare and accurate hypoxemia detection across all racial/ethnic groups.

# Pre Processing

## Handling Missing values

In [ ]:
# Filter Data for Unique Patients to ensure each patient has a unique record
df_filtered = data_train.drop_duplicates(subset='unique_subject_id', keep='first')

Why? The goal is to ensure that each patient is represented by only one record in the dataset. This is important for statistical analyses where each data point should be independent. Multiple records for the same patient can introduce bias and inaccuracies in the analysis.

In [ ]:
# Calculate the percentage of missing values per column
missing_data_percentage = data_train.isnull().mean() * 100
print("Percentage of missing values per column:\n", missing_data_percentage)

- Question: What additional insights can be gained by analyzing missing values beyond just the overall percentage per column?

- Your Answer: ...
- My answer: One of the reasons, by analyzing missing values across different racial/ethnic groups, we can identify potential disparities in data collection. For instance, if certain groups exhibit higher rates of missing data, it may indicate biases or issues in the data gathering process.

In [ ]:

# Calculate percentage of missing values per column and per race

# Get all column names from the DataFrame
all_columns = data_train.columns

# Exclude the 'race_ethnicity' column from the list of all columns
all_columns_ex_race = list(set(all_columns) - set(['race_ethnicity']))

# Create a copy of the original DataFrame to avoid modifying the original data
data_train_missing = data_train.copy()

# Replace non-missing values with 0 and missing values with 1 in the copied DataFrame
# This will allow us to calculate the percentage of missing values
data_train_missing[all_columns_ex_race] = data_train_missing[all_columns_ex_race].isna().astype(int)

# Group the DataFrame by 'race_ethnicity' and calculate the mean of missing values for each group
# This will give the percentage of missing values per column for each race/ethnicity
data_train_missing_by_race = data_train_missing.groupby('race_ethnicity').agg('mean').T

# Display the resulting DataFrame with a background gradient to visualize the percentage of missing values
display(data_train_missing_by_race.style.background_gradient(cmap='inferno'))

- Question: What observations can you make? Are there any columns that we should consider dropping?

- Your answer: ....

In [ ]:
# When you look at the table, given the large % of missing values compared with the other variables - we are dropping coag_fibrinogen and others_ld_ldh
columns_to_drop  = ['coag_fibrinogen', 'others_ld_ldh']
data_train.drop(columns=columns_to_drop, inplace=True)
data_test.drop(columns=columns_to_drop, inplace=True)

- Question: How do you think we should handle missing values? How should we handle missing values for the test set specifically?

- Your answer: ....
- My answer: The goal of this approach is to ensure that missing values are imputed in a way that respects the underlying distribution of the data within each race/ethnicity group. For continuous variables, we use the median to avoid the influence of outliers, while for categorical variables, we use the mode to represent the most frequent category. This method helps maintain the integrity of the data and reduces bias in subsequent analyses.

In [ ]:
def impute_by_race(df, continuous_vars, categorical_vars):
    # Initialize a dictionary to store imputed values for continuous and categorical variables
    impute_values = {'continuous': {}, 'categorical': {}}


    # Loop through each continuous variable to calculate and store the median value by race/ethnicity
    for var in continuous_vars:
        # Calculate the median value for each race/ethnicity group and store it in the dictionary
        impute_values['continuous'][var] = df.groupby('race_ethnicity')[var].median()

        # Impute missing values by filling them with the median value for the respective race/ethnicity group
        df[var] = df.groupby('race_ethnicity')[var].transform(lambda x: x.fillna(x.median()))

    # Loop through each categorical variable to calculate and store the mode (most frequent value) by race/ethnicity
    for var in categorical_vars:
        # Calculate the mode for each race/ethnicity group and store it in the dictionary
        impute_values['categorical'][var] = df.groupby('race_ethnicity')[var].agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan)

        # Impute missing values by filling them with the mode for the respective race/ethnicity group
        df[var] = df.groupby('race_ethnicity')[var].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan))

    # Return the dataframe with imputed values and the dictionary of imputed values
    return df, impute_values

def apply_imputation_for_test_set(test_df, impute_values, continuous_vars, categorical_vars):
    # Loop through each continuous variable to apply the stored imputed values to the test set
    for var in continuous_vars:
        # Loop through each race/ethnicity to apply the respective imputed values
        for race in impute_values['continuous'][var].index:
            # Apply the imputed median value to the missing values in the test set for the respective race/ethnicity group
            test_df.loc[test_df['race_ethnicity'] == race, var] = test_df.loc[test_df['race_ethnicity'] == race, var].fillna(impute_values['continuous'][var][race])

    # Loop through each categorical variable to apply the stored imputed values to the test set
    for var in categorical_vars:
        # Loop through each race/ethnicity to apply the respective imputed values
        for race in impute_values['categorical'][var].index:
            # Apply the imputed mode value to the missing values in the test set for the respective race/ethnicity group
            test_df.loc[test_df['race_ethnicity'] == race, var] = test_df.loc[test_df['race_ethnicity'] == race, var].fillna(impute_values['categorical'][var][race])

    # Return the test dataframe with imputed values
    return test_df

More clarification of the code: The result of the .agg() method is stored in impute_values['categorical'][var]. This will be a Series where the index is the race/ethnicity and the values are the mode of the variable var for each race/ethnicity. In sum, the agg() function is used here to apply the lambda function to each group defined by the 'race_ethnicity' column.



In [ ]:
# Define continuous and categorical variables

categorical_vars = [
    'source_db', 'unique_subject_id', 'unique_hospital_admission_id', 'unique_icustay_id',
    'sex_female', 'race_ethnicity', 'in_hospital_mortality'
]

continuous_vars = [
    'comorbidity_score_value', 'admission_age', 'BMI_admission', 'weight_admission',
    'height_admission', 'sofa_past_overall_24hr', 'sofa_future_overall_24hr',
    'vitals_heart_rate', 'vitals_mbp_ni', 'vitals_resp_rate', 'vitals_tempc',
    'cbc_wbc', 'cbc_hematocrit', 'cbc_platelet', 'coag_inr',
    'bmp_sodium', 'bmp_potassium', 'bmp_chloride', 'bmp_bicarbonate', 'bmp_bun', 'bmp_creatinine',
    'bmp_glucose', 'bmp_aniongap', 'bmp_calcium', 'bmp_lactate', 'hfp_alt', 'hfp_alp', 'hfp_ast',
    'hfp_bilirubin_total', 'hfp_albumin', 'others_ck_cpk',
    'SpO2', 'SaO2', 'pH'
]

In [ ]:
data_train_imputed, impute_values = impute_by_race(data_train, continuous_vars, categorical_vars)
data_test_imputed = apply_imputation_for_test_set(data_test, impute_values, continuous_vars, categorical_vars)

In [ ]:
# Remove rows where target variable 'in_hospital_mortality' is missing
data_train_imputed_final = data_train_imputed.dropna(subset=['in_hospital_mortality'])
data_test_imputed_final = data_test_imputed.dropna(subset=['in_hospital_mortality'])

In [ ]:
# Check for remaining missing values in the imputed training set
missing_data_train = data_train_imputed_final.isnull().sum()
missing_columns_train = missing_data_train[missing_data_train > 0]

if missing_columns_train.empty:
    print("There are no missing values in the training set.")
else:
    print("Columns with remaining missing values in the training set:\n", missing_columns_train)

# Check for remaining missing values in the imputed test set
missing_data_test = data_test_imputed_final.isnull().sum()
missing_columns_test = missing_data_test[missing_data_test > 0]

if missing_columns_test.empty:
    print("There are no missing values in the test set.")
else:
    print("Columns with remaining missing values in the test set:\n", missing_columns_test)


# Exploratory Data Analysis (EDA)

##  Visualization of Mortality per Ethnic Group

- Question: How do you suggest we can do so, and why?
- Your answer: ....

- My answer: Normalizing the data by calculating the mean instead of the sum ensures that
we are comparing the mortality rates relative to the size of each ethnic group.
This is important because it allows us to see the proportional impact of mortality
across different groups, rather than choosing to plot the number of hospital deaths which is skewed by the total number of patients
in each group. This approach helps in identifying if certain groups have a disproportionately
higher or lower mortality rate, providing a clearer insight into potential biases or
disparities in the data.


In [ ]:
# Group by 'ethnicity' and calculate the mean of 'hospital_death' for each group
# This will give the avg mortality rate per ethnical group
mortality_per_ethnicty = data_train_imputed_final.groupby('race_ethnicity')['in_hospital_mortality'].mean()
print (mortality_per_ethnicty)

In [ ]:
# let's plot the results!

# Ensure matplotlib uses an inline backend
%matplotlib inline

# Create the bar plot with a specified figure size
plt.figure (figsize =(10,6))

# Plot the mean mortality per ethnic group as a bar plot
mortality_per_ethnicty.plot(kind = 'bar', color = 'skyblue')

# Add labels and titles to the plot for a better understanding
plt.xlabel('Ethnicity') # Label for the x-axis
plt.ylabel('Mean Hospital Deaths') # Lable for the y-axis
plt.title('Mean Mortality per Ethnic Group') # Title of the plot

# Rotate the x-axis labels for better readability
plt.xticks(rotation = 45)

# Display the plot
plt.show()

## Initial Analysis of SpO2 and SaO2
* Present the statistical summary and visualize the distribution of SpO2 and SaO2 values.
* Highlight any differences in SpO2 and SaO2 distributions across different demographic groups (e.g., race_ethnicity, sex).

Goal: To explore the distribution of SpO2 and SaO2 values across different demographics. This step helps to identify any preliminary differences or patterns in SpO2 and SaO2 measurements among various groups.

In [ ]:
# Statistical summary of SpO2 values
Sp02_summary = data_train_imputed_final['SpO2'].describe()
print("SpO2 summary:\n")
print(Sp02_summary.to_string())
print ("\n")

SaO2_summary = data_train_imputed_final['SaO2'].describe()
print("SaO2 summary:\n")
print(SaO2_summary.to_string())

In [ ]:
# Visualization of the Distribution of SpO2 values
plt.figure(figsize=(16,6))


plt.subplot(1,2,1) # a 1-row, 2-column figure: go to the first subplot
sns.histplot(data_train_imputed_final['SpO2'], kde = True)
plt.title ('Distribution of SpO2 Values')
plt.xlabel('SpO2')
plt.ylabel('Frequency')


# Visualization of the Distribution of SaO2 values
plt.subplot(1,2,2)
sns.histplot(data_train_imputed_final['SaO2'], kde = True)
plt.title ('Distribution of SaO2 Values')
plt.xlabel('SaO2')
plt.ylabel('Frequency')

# Adjust the space between the 2 subplots
plt.subplots_adjust (wspace = 0.2)

plt.show()

- Question: What key insights can you derive from theses plots?

- Your answer: .....


- My answer:
  * High Frequency of 100% SpO2 Readings: The distribution of SpO2 values shows a sharp increase in frequency at the higher end, particularly around 100. This indicates that pulse oximeters frequently report near-100 oxygen saturation levels, suggesting potential issues with overestimation or insensitivity to lower oxygen saturation levels.

  * Wider Distribution and Lower Peak for SaO2 Values: The SaO2 distribution is broader and does not show the same sharp peak at the higher end as the SpO2 distribution. This suggests that the more accurate arterial blood gas measurements capture a wider range of oxygen saturation levels and are less likely to overestimate saturation, highlighting the discrepancy between noninvasive and invasive measurements.

### Comparison of Arterial Oxygen Saturation by Pulse Oximetry Readings

In [ ]:
# Create a copy of the relevant columns from the original DataFrame, filtering only 'White' and 'Black' races
df = data_train_imputed_final.copy()[['SpO2','SaO2','race_ethnicity']][data_train_imputed_final['race_ethnicity'].isin(['White','Black'])]

# Import the linregress function from scipy.stats for performing linear regression, and t for statistical tests
from scipy.stats import linregress, t

# Create a new DataFrame that only includes rows where SpO2 is greater than or equal to 89
df89 = df.copy()[df.SpO2 >= 89]

# Group the DataFrame by 'race_ethnicity'
races = df89.groupby('race_ethnicity')

# Create a matplotlib figure and axes for plotting
fig, ax = plt.subplots()

# Define the x-value where we want to calculate the intercept
x_start = 89

# Define offsets for plotting each race separately to avoid overlapping points
offsets = {'White': -0.2, 'Black': 0.}

# Loop through each group (race) and perform linear regression
for name, race in races:
    # Perform linear regression between SpO2 and SaO2 for the current race group
    slope, intercept, r_value, p_value, std_err = linregress(race['SpO2'], race['SaO2'])

    # Calculate the intercept at x = 89 using the regression line equation
    intercept_at_x_start = slope * x_start + intercept

    # Plot the regression line with an offset for each race group to avoid overlapping points
    offset_x = race['SpO2'] + offsets[name]
    ax.plot(offset_x, race['SaO2'], 'o', label=f'Race {name}')
    ax.plot(race['SpO2'], slope * race['SpO2'] + intercept, label=f'Race {name} fit: y={slope:.2f}(x - {x_start} )+{intercept_at_x_start:.2f}')

    # Print the slope and intercept for each race group
    print(f'Race {name}:')
    print(f'  Slope: {slope:.2f}')
    print(f'  Intercept at x={x_start}: {intercept_at_x_start:.2f}')
    print()


# Add a legend to the plot to distinguish between the different races
ax.legend()

# Label the x-axis as 'SpO2'
plt.xlabel('SpO2')

# Label the y-axis as 'SaO2'
plt.ylabel('SaO2')

# Set the title of the plot
plt.title('Regression Lines for Different Races')

# Display the plot
plt.show()






- Question: What key insights can you derive from this plot?
- Your answer:
- My answer: The plot suggests that Black patients generally have lower oxygen saturation levels (SaO2) compared to White patients, even when their SpO2 readings are the same. This indicates a disparity where the same SpO2 value corresponds to a lower SaO2 for Black patients than for White patients.



## Analysis of Hypoxemia using SpO2 and SaO2 individually

* Define hypoxemia using a threshold for SpO2 and SaO2.
* Compare the incidence of hypoxemia across different demographic groups.
* Highlight any disparities that may indicate bias in SpO2 measurements.

Goal: To compare SpO2 and SaO2 values side by side and examine the incidence of hypoxemia using both measurements. This step highlights the discrepancies and biases in SpO2 measurements by analyzing hypoxemia based on SaO2 measurements and compare it with SpO2-based analysis. This step emphasizes the importance of SaO2 measurements in revealing hidden hypoxemia that SpO2 alone might miss.

In [ ]:


# Create a copy of the DataFrame
data_train_imputed_final_copy = data_train_imputed_final.copy()


# Define Hypoxemia based on SpO2 threshold (e.g. SpO2 < 88%) and SpO2 threshold (e.g. SpO2 < 88%)
data_train_imputed_final_copy ['hypoxemia_SpO2'] = data_train_imputed_final_copy ['SpO2'] < 88
data_train_imputed_final_copy ['hypoxemia_SaO2'] = data_train_imputed_final_copy ['SaO2'] < 88

# Incidence of hypoxemia by race_ethnicity
hypoxemia_by_race_SpO2 = data_train_imputed_final_copy.groupby('race_ethnicity')['hypoxemia_SpO2'].mean().reset_index()
hypoxemia_by_race_SaO2 = data_train_imputed_final_copy.groupby('race_ethnicity')['hypoxemia_SaO2'].mean().reset_index()
    # The mean value represents the proportion of patients with hypoxemia in each racial/ethnic group.


hypoxemia_by_race_SpO2.columns = ['race_ethnicity', 'hypoxemia_incidence']
hypoxemia_by_race_SaO2.columns = ['race_ethnicity', 'hypoxemia_incidence']
hypoxemia_by_race_SpO2['hypoxemia_incidence'] *= 100
hypoxemia_by_race_SaO2['hypoxemia_incidence'] *= 100



print("SpO2:\n")
print (hypoxemia_by_race_SpO2)
print('\n')
print("SaO2:\n")
print (hypoxemia_by_race_SaO2)


### Hypoxemia Incidence

In [ ]:
# Visualization
plt.figure(figsize=(12,8))


# Define the common y-axis limit
y_max = max(hypoxemia_by_race_SpO2['hypoxemia_incidence'].max(), hypoxemia_by_race_SaO2['hypoxemia_incidence'].max())
# y_min = min(hypoxemia_by_race_SpO2['hypoxemia_incidence'].min(), hypoxemia_by_race_SaO2['hypoxemia_incidence'].min())

# a 1-row, 2-column figure: go to the first subplot
plt.subplot(1,2,1)
sns.barplot(x='race_ethnicity', y='hypoxemia_incidence', data = hypoxemia_by_race_SpO2)
plt.title ('Hypoxemia Incidence - SpO2')
plt.xlabel('Race/Ethnicity')
plt.ylabel ('Hypoxemia Incidence (%)')
plt.xticks(rotation = 45)
plt.ylim(0,y_max)


# a 1-row, 2-column figure: go to the second subplot
plt.subplot(1,2,2)
sns.barplot(x='race_ethnicity', y='hypoxemia_incidence', data = hypoxemia_by_race_SaO2)
plt.title ('Hypoxemia Incidence - SaO2')
plt.xlabel('Race/Ethnicity')
plt.ylabel ('Hypoxemia Incidence (%)')
plt.xticks(rotation = 45)
plt.ylim(0,y_max)

# Adjust the space between the 2 subplots
plt.subplots_adjust (wspace = 0.2)

plt.show()

- Question: What key insights can you derive from theses plots?
- Your answer: ....


- My answer:
  * The disparity between the incidence of hypoxemia based on SpO2 and SaO2 measurements  indicates that relying only on SpO2 can lead to underestimation of hypoxemia, especially in certain racial-ethnic groups.
  * The analysis reveals that SaO2 measurements uncover hidden hypoxemia cases not detected by SpO2 alone.

## Hidden Hypoxemia
This brings us to the concept of hidden hypoxemia. Hidden hypoxemia occurs when patients exhibit low blood oxygen levels that are not detected by pulse oximetry. The accuracy issues and dataset imbalances can exacerbate this problem, particularly in non-white populations. In the next section, we will explore the prevalence of hidden hypoxemia across different racial and ethnic groups, highlighting the critical need for accurate and equitable diagnostic tools in clinical settings.


* Identify hidden hypoxemia using both SpO2 and SaO2 measurements.
* Compare the incidence of hidden hypoxemia across different demographic groups.
* Highlight how the bias in SpO2 measurements can lead to hidden hypoxemia, particularly in certain demographic groups.

Goal: To identify hidden hypoxemia (patients with normal SpO2 but low SaO2) and analyze its prevalence across different demographics. This step uncovers the hidden bias in SpO2 measurements and shows how certain groups are disproportionately affected.


### Hidden Hypoxemia - SpO2 & SaO2

In [ ]:

data_test_imputed_final['hidden_hypoxemia'] = data_test_imputed_final.apply(lambda row: row['SpO2'] >= 88 and row['SaO2'] < 88, axis=1)


# Identify hidden hypoxemia (SpO2 >= 88% and SaO2 < 88%)
data_train_imputed_final['hidden_hypoxemia'] = data_train_imputed_final.apply(lambda row: row['SpO2'] >= 88 and row['SaO2'] < 88, axis=1)

# Incidence of hidden hypoxemia by race_ethnicity
hidden_hypoxemia_by_race = data_train_imputed_final.groupby('race_ethnicity')['hidden_hypoxemia'].mean().reset_index()
hidden_hypoxemia_by_race.columns = ['race_ethnicity', 'hidden_hypoxemia_incidence']
hidden_hypoxemia_by_race['hidden_hypoxemia_incidence'] *= 100

print(hidden_hypoxemia_by_race)

In [ ]:
 # Visualization
plt.figure(figsize=(12, 8))
sns.barplot(x='race_ethnicity', y='hidden_hypoxemia_incidence', data=hidden_hypoxemia_by_race)
plt.title('Hidden Hypoxemia Incidence by Race/Ethnicity')
plt.xlabel('Race/Ethnicity')
plt.ylabel('Hidden Hypoxemia Incidence (%)')
plt.xticks(rotation = 45)

plt.show()



- Question: What key insights can you derive from this plot?
- Your answer: ....


- My answer: The higher incidence of hidden hypoxemia in Black patients further supports the notion of bias in SpO2 measurements. Pulse oximeters may be less accurate for individuals with darker skin, resulting in normal SpO2 readings despite low SaO2 levels. This can lead to missed diagnoses and delayed treatment for hypoxemia.

### Conclusion:

* Objective: To identify hidden hypoxemia by considering both SpO2 and SaO2 measurements and to compare its incidence across different racial/ethnic groups.
* Findings: The analysis of hidden hypoxemia shows even more pronounced disparities, with Black patients having the highest incidence.
* Implication: The high incidence of hidden hypoxemia in Black patients indicates that relying solely on SpO2 can miss cases of low blood oxygen levels, particularly in this group. This highlights the importance of considering SaO2 measurements to ensure accurate diagnosis and treatment.

The analysis highlights the importance of considering both SpO2 and SaO2 measurements in clinical settings, especially for patients from racial/ethnic groups that may be more susceptible to inaccuracies in SpO2 readings.

# Modeling

## SpO2 modifications - exaggerating the bias
This first step is important in the tutorial as it simulates the impact of a known bias in medical devices, specifically pulse oximeters, which often overestimate blood oxygen levels in Black patients. By artificially increasing SpO2 values for Black patients by 10%, we create a dataset that exaggerates this bias. This allows us to analyze how such inaccuracies affect downstream clinical predictions, such as mortality, and highlights the need for fair and accurate data in healthcare machine learning models.


In [ ]:
# Display the baseline distribution of SpO2
print("Baseline SpO2 Distribution:")
print(data_train_imputed_final['SpO2'].describe())

In [ ]:
# Add bias to Black patients' SpO2
delta_to_add = 10

data_train_imputed_final['SpO2_exaggerated'] = data_train_imputed_final.apply(
    lambda row: min(row['SpO2'] + delta_to_add, 100) if row['race_ethnicity'] == 'Black' else row['SpO2'],
    axis=1
)


data_test_imputed_final['SpO2_exaggerated'] = data_test_imputed_final.apply(
    lambda row: min(row['SpO2'] + delta_to_add, 100) if row['race_ethnicity'] == 'Black' else row['SpO2'],
    axis=1
)

In [ ]:
# Compare both distributions for Black patients before and after modification
print("Before modification:")
print(data_train_imputed_final.loc[data_train_imputed_final['race_ethnicity'] == 'Black', 'SpO2'].describe())

print ("\n")
print("After modification:")
print(data_train_imputed_final.loc[data_train_imputed_final['race_ethnicity'] == 'Black', 'SpO2_exaggerated'].describe())


There is a 1% difference in the median which shows the impact of the introduced bias.
N.B: The median is derived from the .describe() function, which provides various statistical summaries of the data, including the median (50% value).

In [ ]:
data_train_imputed_final.columns

In [ ]:
# Check for remaining missing values in the imputed training set
missing_data_train = data_train_imputed_final.isnull().sum()
missing_columns_train = missing_data_train[missing_data_train > 0]

if missing_columns_train.empty:
    print("There are no missing values in the training set.")
else:
    print("Columns with remaining missing values in the training set:\n", missing_columns_train)

## Encoding

Since the machine learning model in the background models and finds patterns in our data. It only supports numeric values. For this reason, categorical variables must be coded to numeric values.

Categorical variables can be of 3 types:
* Binary variables: Binary variables can be represented with two values, 1 and 0. Examples are whether or not the variable belongs to a group.
* Ordinal variables: Ordinal variables are a type of variables that have a specific order and can be represented with numeric variables through a label encoder. An example is High, Medium, and Low which can be represented as 3, 2, 1.
* Nominal variables: Nominal variables are categorical variables that do not have a defined order, for these variables it is not recommended to use a label encoder, it is better to use one hot encoder in these cases.

In [ ]:
unique_values = data_train_imputed_final.apply(lambda x: x.unique())

In [ ]:
unique_values

## Standardization
- Why Standardization?
Standardization is generally preferred when dealing with datasets that contain outliers, features with different scales, or when the model assumptions align better with standardized data. It helps to reduce the influence of outliers, stabilize model coefficients, and improve the performance and interpretability of many machine learning algorithms. Normalization, on the other hand, is useful when the absolute scale of the features is important or when using algorithms that are sensitive to the range of the data.

- Remember:
    * When features are not on similar scales, some algorithms may be more heavily influenced by certain features than others, which can lead to suboptimal performance. Additionally, some algorithms (such as those based on distance calculations) can be sensitive to differences in scale between features, which can lead to incorrect results.
    * By standardizing the data, we can ensure that each feature contributes equally to the model, regardless of its scale. This can lead to better accuracy and more robust models.



Note: Use the function to normalize the data. Remember to exclude information that does not contribute to the final model, such as identifiers.

In [ ]:
data_train_imputed_final = data_train_imputed_final.drop(['sex'], axis=1)

In [ ]:
# Define a function standardized_data

def standardized_data(train_data, test_data=None, scaler=None, ignore_cols=[]):

    train_data_filtered = train_data

    # Drop columns that should not be standardized (e.g., identifiers)
    train_data_filtered = train_data_filtered.drop(ignore_cols, axis=1, errors='ignore')
    if test_data is not None:
        test_data_filtered = test_data.drop(ignore_cols, axis=1, errors='ignore')

    # If no scaler is provided, initialize a StandardScaler
    if not scaler:
        scaler = StandardScaler()
        scaler.fit(train_data_filtered)  # Fit the scaler on the training data

    # Transform the training data using the scaler
    standardized_train = scaler.transform(train_data_filtered)

    # Convert the standardized data back to a DataFrame with the same columns as train_data_filtered
    standardized_train_df = pd.DataFrame(standardized_train, columns=train_data_filtered.columns)

    # If test data is provided, transform it using the same scaler
    if test_data is not None:
        # Normalize the test data
        standardized_test = scaler.transform(test_data_filtered)
                ##### note here - to ensure there is no data leakage we are using the scaler fitted on the training data to transform the test data (so the test data stays protected)
        # Convert the standardized test data back to a DataFrame with the same columns as test_data_filtered
        standardized_test_df = pd.DataFrame(standardized_test, columns=test_data_filtered.columns)

        # Return the standardized training and test data along with the scaler
        return standardized_train_df, standardized_test_df, scaler
    else:
        # If no test data is provided, return only the standardized training data and the scaler
        return standardized_train_df, scaler

Note: Why Return the Scaler? Returning the scaler allows you to use the same scaler for any future data transformations, ensuring consistency. This is especially important if you need to apply the same scaling to new data or to reverse the scaling transformation.

How to know what columns to include? You should exclude identifiers and categorical variables that are not intended to be scaled

In [ ]:
# Define the columns to ignore during normalization (identifiers and categorical variables)
ignore_cols = [
    'unique_subject_id', 'unique_hospital_admission_id', 'unique_icustay_id',
    'source_db', 'sex_female', 'race_ethnicity', 'in_hospital_mortality', 'hidden_hypoxemia'
]

In [ ]:
from sklearn.preprocessing import StandardScaler


final_standardized_train, final_standardized_test, scaler = standardized_data(
    data_train_imputed_final, data_test_imputed_final, ignore_cols=ignore_cols
)

# Selecting ignored columns
ignored_columns_train = data_train_imputed_final[ignore_cols]
ignored_columns_test = data_test_imputed_final[ignore_cols]

# Reset indices to ensure alignment before concatenation
ignored_columns_train = ignored_columns_train.reset_index(drop=True)
final_standardized_train = final_standardized_train.reset_index(drop=True)

ignored_columns_test = ignored_columns_test.reset_index(drop=True)
final_standardized_test = final_standardized_test.reset_index(drop=True)

# Merge the ignored columns with the standardized columns
final_standardized_train = pd.concat([ignored_columns_train, final_standardized_train], axis=1)
final_standardized_test = pd.concat([ignored_columns_test, final_standardized_test], axis=1)


# Display the shapes of the DataFrames for verification
print("Final standardized train shape:", final_standardized_train.shape)
print("Final standardized test shape:", final_standardized_test.shape)

## Defining our X_train, y_train, X_test, y_test


In [ ]:
data_train_imputed_final.groupby("race_ethnicity")['in_hospital_mortality'].describe() # to see how many races in our X train

In [ ]:
# List of columns to exclude from the feature matrix
exclude_cols = [
    'sofa_future_overall_24hr', 'sofa_past_overall_24hr',
    'unique_subject_id', 'unique_hospital_admission_id',
    'unique_icustay_id', 'source_db', 'race_ethnicity', 'in_hospital_mortality'
]

# Define the feature matrix X and target variable y for training and test sets
X_train = final_standardized_train.drop(exclude_cols, axis=1)
y_train = final_standardized_train['in_hospital_mortality']

X_test = final_standardized_test.drop(exclude_cols, axis=1)
y_test = final_standardized_test['in_hospital_mortality']

# Check lengths and indices
print(f"Length of X_train: {len(X_train)}")
print(f"Length of y_train: {len(y_train)}")
print(f"X_train indices: {X_train.index}")
print(f"y_train indices: {y_train.index}")

## Step 1: Understanding the influence of hidden hypoxemia on patient outcomes
* Following the paper's approach, we are going to perform univariate and multivariate logistic regression to assess binary endpoints (e.g., in-hospital mortality) while adjusting for other covariates (e.g., age, sex, SOFA score), and to use analysis of variance to test for the impact of hidden hypoxemia.

* Goal: Assess the impact of hidden hypoxemia by analyzing its effect on patient outcomes like in-hospital mortality.

### Univariate Logistic Regression

In [ ]:
import statsmodels.api as sm  # Add statsmodels for logistic regression
import numpy as np
import pandas as pd

# List of variables to analyze
vars = ['admission_age',
        'sex_female',
        'BMI_admission',
        'vitals_heart_rate', 'vitals_mbp_ni',
        'vitals_resp_rate', 'vitals_tempc',
        'cbc_wbc', 'cbc_hematocrit',
        'cbc_platelet', 'coag_inr', 'bmp_sodium',
        'bmp_potassium', 'bmp_chloride', 'bmp_bicarbonate', 'bmp_bun',
        'bmp_creatinine', 'bmp_glucose', 'bmp_aniongap', 'bmp_calcium',
        'bmp_lactate', 'hfp_alt', 'hfp_alp', 'hfp_ast', 'hfp_bilirubin_total',
        'hfp_albumin', 'others_ck_cpk',
        'SpO2', 'SaO2', 'pH',
        'SpO2_exaggerated', 'hidden_hypoxemia']

# Initialize an empty DataFrame to store the results
results_df = pd.DataFrame()

X_train['hidden_hypoxemia'] = X_train['hidden_hypoxemia'].astype(int)

# Iterate over each variable in the list
for var in vars:
    X = X_train[var]  # Select the current variable from the training data
    y = y_train       # Target variable remains the same

    # statsmodels must explicitly add a constant term
    # Add a constant term to the independent variable for the logistic regression model
    X = sm.add_constant(X)

    # Application of logistic regression model using statsmodels
    model = sm.Logit(y, X)
    result = model.fit(disp=False)  # Fit the model without printing the summary

    # Calculation of p-values and odds ratios
    # Extract the summary of the fitted model
    summary = result.summary2()

    # Extract p-values from the summary
    p_values = summary.tables[1]['P>|z|']

    # Calculate the odds ratios by exponentiating the coefficients
    odds_ratios = np.exp(summary.tables[1]['Coef.'])

    # Calculate the 95% confidence intervals for the odds ratios
    conf_int = np.exp(summary.tables[1][['[0.025', '0.975]']])

    # Create a temporary DataFrame to store the results for the current variable
    results_df_ = pd.DataFrame({
        'Variable': summary.tables[1].index,  # Variable names
        'P-value': p_values,                  # P-values
        'Odds Ratio': odds_ratios,            # Odds ratios
        '95% CI (Lower)': conf_int['[0.025'], # Lower bound of the 95% confidence interval
        '95% CI (Upper)': conf_int['0.975]']  # Upper bound of the 95% confidence interval
    })

    # Filter the results to include only the current variable and concatenate to the final results DataFrame
    results_df = pd.concat([results_df, results_df_[results_df_['Variable'] == var]], axis=0)

# Display the final results DataFrame
display(results_df)

In [ ]:
# Create a figure and set the size
plt.figure(figsize=(8, 6))

# Create a horizontal bar plot of the p-values for each variable
plt.barh(results_df['Variable'], results_df["P-value"], label='Exponential Growth')  # Draw a horizontal bar graph

# Set the x-axis to a logarithmic scale
plt.xscale('log')  # Set the x-axis to a logarithmic scale
plt.gca().invert_yaxis()  # Invert the order of the y-axis (change from top to bottom)

# Decorate the graph
plt.title('Exponential p_value on Log Scale')  # Set the title of the graph
plt.xlabel('X-axis (log scale)')  # Change the label of the x-axis
plt.ylabel('Y-axis')  # Set the label of the y-axis
plt.grid(True)  # Enable grid
plt.legend()  # Display the legend

# Display the graph
plt.show()  # Show the plot


- Question: What key insights can you derive from this figure?
- Your answer: ....


- My answer: The figure effectively highlights the statistical significance of various clinical variables, with lactate standing out as the most important. However, it also underscores the limitations of univariate analysis in capturing the full clinical picture, particularly for complex conditions like hidden hypoxemia. For a more comprehensive understanding, a multivariate analysis would be necessary to consider the interactions between these variables and their collective impact on patient outcomes.

### Multivariate Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

# Define the features to include
features_to_include = ['admission_age',
        'sex_female',
        'BMI_admission',
        'vitals_heart_rate', 'vitals_mbp_ni',
        'vitals_resp_rate', 'vitals_tempc',
        'cbc_wbc', 'cbc_hematocrit',
        'cbc_platelet', 'coag_inr', 'bmp_sodium',
        'bmp_potassium', 'bmp_chloride', 'bmp_bicarbonate', 'bmp_bun',
        'bmp_creatinine', 'bmp_glucose', 'bmp_aniongap', 'bmp_calcium',
        'bmp_lactate', 'hfp_alt', 'hfp_alp', 'hfp_ast', 'hfp_bilirubin_total',
        'hfp_albumin', 'others_ck_cpk',
        'pH', 'hidden_hypoxemia']

# Fit the logistic regression model
model = LogisticRegression (max_iter = 1000)
result = model.fit(X_train[features_to_include], y_train)

# Predictions
y_pred = model.predict(X_test[features_to_include])
y_pred_proba = model.predict_proba(X_test[features_to_include]) [:,1]


# Evaluate the model
print("Classification Report:")
print (classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")

# Area under the Curve (AUC)
auc = roc_auc_score(y_test, y_pred_proba)
print(f"AUC: {auc:.2f}")

In [ ]:
# Extract coefficients and their importance
coefficients = model.coef_[0]
importance = pd.DataFrame({'Feature': X_train[features_to_include].columns, 'Importance': coefficients})
importance = importance.sort_values(by='Importance', ascending=False)

# Plotting
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=importance)
plt.title('Feature Importance in Logistic Regression')
plt.show()

- Question: What key insights can you derive from this figure?
- Your answer: ....


- My answer: The multivariate analysis provides a more nuanced view of feature importance, demonstrating how variables interact with each other to influence outcomes. Hidden hypoxemia, which was not significant in the univariate analysis, emerges as highly important in the multivariate context, emphasizing the value of considering multiple variables simultaneously. This approach provides a more comprehensive understanding of the factors affecting patient outcomes and helps identify the key predictors in a clinical setting.


#### *Optional*
 * Task: We already assessed the importance of hidden hypoxemia on in-hospital mortality as a binary outcome, now try to:
   - Assess the importance of hidden hypoxemia on ordinal outcomes, such as CVSOFA (Cardiovascular Sequential Organ Failure Assessment) and RSOFA (Respiratory Sequential Organ Failure Assessment) scores.
   - Assess the importance of hidden hypoxemia on continuous outcomes, such as creatinine and lactate levels.
  
 * What observations do you see?

## Step 2: Predicting In Hospital Mortality using different features in the dataset
Goal: Compare the performance of different models that include various combinations of features (SpO2, Sp02_exaggerated, SaO2) in predicting in-hospital mortality across different patient races. This step demonstrates how including or excluding certain features impacts model accuracy, fairness, and overall predictive performance, emphasizing the need for careful feature selection in clinical predictions.




In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score


# Helper function to train and evaluate models
def train_model (X_train, X_test, y_train, features):
    # Create an instance of Logistic Regression model with a maximum of 1000 iterations for convergence
    model = LogisticRegression(max_iter=1000)

    # Fit the model on the training data using the specified features
    model.fit(X_train[features], y_train)

    # Predict the target values for the test data
    y_pred = model.predict(X_test[features])

    # Predict the probabilities of the positive class for the test data
    y_pred_proba = model.predict_proba(X_test[features])[:, 1]

    # Return the trained model, predictions, predicted probabilities
    return model, y_pred, y_pred_proba


In [ ]:
# Define feature sets dynamically excluding 'hidden_hypoxemia'
all_features = X_train.columns.tolist()

# Exclude 'SpO2_exaggerated', 'SpO2', and 'hidden_hypoxemia' for features_sao2
features_sao2 = [feature for feature in all_features if feature not in ['SpO2_exaggerated', 'SpO2', 'hidden_hypoxemia']]

# Exclude 'SaO2', 'SpO2_exaggerated', and 'hidden_hypoxemia' for features_spo2
features_spo2 = [feature for feature in all_features if feature not in ['SaO2', 'SpO2_exaggerated', 'hidden_hypoxemia']]

# Exclude 'SaO2', 'SpO2', and 'hidden_hypoxemia' for features_exaggerated_spo2
features_exaggerated_spo2 = [feature for feature in all_features if feature not in ['SaO2', 'SpO2', 'hidden_hypoxemia']]


In [ ]:
# Train each model
model_sao2, y_pred_sao2, y_pred_proba_sao2 = train_model(X_train, X_test, y_train, features_sao2)
model_spo2, y_pred_spo2, y_pred_proba_spo2 = train_model(X_train, X_test, y_train, features_spo2)
model_exaggerated_spo2, y_pred_exaggerated_spo2, y_pred_proba_exaggerated_spo2 = train_model(X_train, X_test, y_train, features_exaggerated_spo2)

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, balanced_accuracy_score

# Evaluate across races
def evaluate_across_races(X_test, y_test, y_pred, y_pred_proba, data_test, model, features):
    """
    Evaluate model performance across different race/ethnicity groups.

    This function evaluates the performance of a trained machine learning model across
    different race/ethnicity groups in the test dataset. It generates classification reports,
    calculates accuracy and AUC scores, and identifies feature importance for each group.

    Parameters:
    X_test (array-like): Test features.
    y_test (array-like): True labels for the test set.
    y_pred (array-like): Predicted labels for the test set.
    y_pred_proba (array-like): Predicted probabilities for the positive class.
    data_test (DataFrame): Test dataset containing the race/ethnicity information.
    model (sklearn model): Trained machine learning model.
    features (list): List of feature names.

    Returns:
    group_reports (dict): Dictionary containing classification reports, accuracy, and AUC scores for each race/ethnicity group.
    feature_importance_dict (dict): Dictionary containing feature importance for each race/ethnicity group.
    """

    # Create a DataFrame for evaluation using the test features
    evaluation_df = pd.DataFrame(X_test, columns=features)

    # Add the actual test labels to the DataFrame
    evaluation_df['y_test'] = y_test.values

    # Add the predicted labels to the DataFrame
    evaluation_df['y_pred'] = y_pred

    # Add the predicted probabilities to the DataFrame
    evaluation_df['y_pred_proba'] = y_pred_proba

    # Add the race/ethnicity information to the DataFrame
    evaluation_df['race_ethnicity'] = data_test['race_ethnicity'].values



    # Initialize dictionaries to store reports and feature importances for each race group
    group_reports = pd.DataFrame()

    # Loop through each unique race/ethnicity group in the evaluation DataFrame
    for i, group in enumerate(evaluation_df['race_ethnicity'].unique()):
        # Filter the data for the current race/ethnicity group
        group_data = evaluation_df[evaluation_df['race_ethnicity'] == group]

        # Extract the actual labels for the current group
        y_test_group = group_data['y_test']

        # Extract the predicted labels for the current group
        y_pred_group = group_data['y_pred']

        # Extract the predicted probabilities for the current group
        y_pred_proba_group = group_data['y_pred_proba']


        # Generate a classification report for the current group
        report = classification_report(y_test_group, y_pred_group, output_dict=True)

        group_reports.loc[i, 'race_ethnicity'] = group

        # Weighted precision for the race group
        group_reports.loc[i, "Weighted Precision"] = report['weighted avg']['precision']

        # Weighted recall for the race group
        group_reports.loc[i, "Weighted Recall"] = report['weighted avg']['recall']

        # Weighted F1-score for the race group
        group_reports.loc[i, "Weighted F1-Score"] = report['weighted avg']['f1-score']

        # Calculate the balanced accuracy for the current group
        group_reports.loc[i, 'balanced_accuracy'] = balanced_accuracy_score(y_test_group, y_pred_group)

        # Calculate the AUC score for the current group
        group_reports.loc[i, 'auc'] = roc_auc_score(y_test_group, y_pred_proba_group)

    return group_reports




In [ ]:
# Evaluate across races for each model
group_reports_sao2 = evaluate_across_races(X_test, y_test, y_pred_sao2, y_pred_proba_sao2, final_standardized_test, model_sao2, features_sao2)

group_reports_spo2 = evaluate_across_races(X_test, y_test, y_pred_spo2, y_pred_proba_spo2, final_standardized_test, model_spo2, features_spo2)

group_reports_exaggerated_spo2 = evaluate_across_races(X_test, y_test, y_pred_exaggerated_spo2, y_pred_proba_exaggerated_spo2, final_standardized_test, model_exaggerated_spo2, features_exaggerated_spo2)


In [ ]:
group_reports_sao2['features'] = 'SaO2'
group_reports_spo2['features'] = 'SpO2'
group_reports_exaggerated_spo2['features'] = 'Exaggerated SpO2'

all_reports = pd.concat([group_reports_sao2,group_reports_spo2,group_reports_exaggerated_spo2],axis=0)

all_reports.sort_values(['race_ethnicity','features'])[['features','race_ethnicity', 'auc', 'Weighted Precision', 'Weighted Recall',
       'Weighted F1-Score', 'balanced_accuracy']]

Reminder:
- Recall: This is critical in your context because it measures the ability of the model to identify all actual positive cases (e.g., patients who died). High recall ensures that the model doesn't miss many critical cases, which is essential in a healthcare setting.

- Precision: While not as critical as recall, precision indicates the proportion of positive identifications that are actually correct. It helps in understanding the false positive rate.

- F1-Score: This metric provides a balance between precision and recall. It's useful when you want a single metric that considers both false positives and false negatives.

- Accuracy: While it gives an overall performance measure, it can be misleading in imbalanced datasets (e.g., more patients survive than die).

- AUC (Area Under the Curve): This metric helps understand the model's ability to distinguish between positive and negative cases across various threshold settings. It’s useful for evaluating overall model performance.



### Calibration Plot

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve
from sklearn.linear_model import LogisticRegression

# Calibration curve for model_sao2
prob_true_sao2, prob_pred_sao2 = calibration_curve(y_test, y_pred_proba_sao2, n_bins=10)

# Calibration curve for model_spo2
prob_true_spo2, prob_pred_spo2 = calibration_curve(y_test, y_pred_proba_spo2, n_bins=10)

# Calibration curve for model_exaggerated_spo2
prob_true_exaggerated_spo2, prob_pred_exaggerated_spo2 = calibration_curve(y_test, y_pred_proba_exaggerated_spo2, n_bins=10)

# Plot calibration curves
plt.figure(figsize=(10, 7))

# Plot calibration curve for model_sao2
plt.plot(prob_pred_sao2, prob_true_sao2, marker='o', label='Model SAO2', color='blue', linewidth=0.5)

# Plot calibration curve for model_spo2
plt.plot(prob_pred_spo2, prob_true_spo2, marker='o', label='Model SPO2', color='orange', linewidth=0.5)

# Plot calibration curve for model_exaggerated_spo2
plt.plot(prob_pred_exaggerated_spo2, prob_true_exaggerated_spo2, marker='o', label='Model Exaggerated SPO2', color='green', linewidth=0.5)

# Plot perfectly calibrated line
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Perfectly Calibrated', linewidth=1)


plt.xlabel('Predicted Probability')
plt.ylabel('True Probability')
plt.title('Calibration Curves')
plt.legend()
plt.show()


- Question: What key insights can you derive from this figure?
- Your answer: ....


- My answer: The calibration curves indicate that all three models (SAO2, SPO2, and Exaggerated SPO2) are well-calibrated, providing reliable probability estimates. Minor deviations at higher probability levels are observed but are not significant enough to impact the overall reliability of the models. This suggests that these models are suitable for use in clinical settings where accurate probability predictions are essential for decision-making.

#### *Optional*
 * Try the same approach to assess different outcomes:
   - Ordinal outcomes, such as CVSOFA (Cardiovascular Sequential Organ Failure Assessment) and RSOFA (Respiratory Sequential Organ Failure Assessment) scores.
   - Continuous outcomes, such as creatinine and lactate levels.
  
 * What observations do you see?

Acknowledgement: Thank you to Dr. Leo Anthony Celi, Dr. Takeshi Tohyama, João Matos, and many others from Dr. Celi’s team at [MIT Critical Data](https://criticaldata.mit.edu/#community) for their support throughout the development of this notebook.